In [ ]:
import vertexai
from vertexai import agent_engines
from agent import root_agent # modify this if your agent is not in agent.py
import os

from dotenv import load_dotenv, dotenv_values, set_key

In [ ]:
load_dotenv()

PROJECT_ID = os.environ["GOOGLE_CLOUD_PROJECT"]
LOCATION = os.environ["GOOGLE_CLOUD_LOCATION"]
STAGING_BUCKET = os.environ["STAGING_BUCKET"]
AGENT_DISPLAY_NAME = os.environ["AGENT_DISPLAY_NAME"]

app = agent_engines.AdkApp(
    agent=root_agent,
    enable_tracing=True,
)

In [ ]:
print(f"PROJECT_ID: {PROJECT_ID}")
print(f"LOCATION: {LOCATION}")
print(f"STAGING_BUCKET: {STAGING_BUCKET}")

In [ ]:
vertexai.init(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=STAGING_BUCKET,
)

## Roda local

Vamos primeiro verificar se está funcionando e carregamos a versão correta

In [ ]:
query = ""
user_id="u_123"

In [ ]:
events = []
async for event in app.async_stream_query(
    user_id=user_id,
    message=query,
):
    events.append(event)

# The full event stream shows the agent's thought process
#print("--- Full Event Stream ---")
#for event in events:
#    print(event)

# For quick tests, you can extract just the final text response
final_text_responses = [
    e for e in events
    if e.get("content", {}).get("parts", [{}])[0].get("text")
    and not e.get("content", {}).get("parts", [{}])[0].get("function_call")
]
if final_text_responses:
    print("\n--- Final Response ---")
    print(final_text_responses[0]["content"]["parts"][0]["text"])

## Deploy

In [ ]:
remote_app = agent_engines.create(
    app,
    requirements="requirements.txt",
    extra_packages=["agent.py"],
    display_name=AGENT_DISPLAY_NAME,
    env_vars=dotenv_values("../.env")
)

In [ ]:
resource_name = remote_app.resource_name
print(f"Resource name: {resource_name}")

In [ ]:
set_key(dotenv_path="../.env", key_to_set="REASONING_ENGINE_ID", value_to_set=resource_name.split('/')[-1])

## Teste Agent Engine

Obs: Não vai encontrar o documento se precisar de Autenticação, como gDrive. Vamos apenas testar o endpoint

In [ ]:
AGENT_DISPLAY_NAME = os.environ["GOOGLE_CLOUD_PROJECT_NUMBER"]

In [ ]:
if 'resource_name' not in locals():
    REASONING_ENGINE_ID = os.environ["REASONING_ENGINE_ID"]
    PROJECT_NUMBER = os.environ["GOOGLE_CLOUD_PROJECT_NUMBER"]
    resource_name = f"projects/{PROJECT_NUMBER}/locations/us-central1/reasoningEngines/{REASONING_ENGINE_ID}"

print(f"Resource name: {resource_name}")
remote_agent = agent_engines.get(resource_name)

In [ ]:
query = ""
user_id="u_123"

In [ ]:
events = []
async for event in remote_agent.async_stream_query(
    user_id=user_id,
    message=query,
):
    events.append(event)


# For quick tests, you can extract just the final text response
final_text_responses = [
    e for e in events
    if e.get("content", {}).get("parts", [{}])[0].get("text")
    and not e.get("content", {}).get("parts", [{}])[0].get("function_call")
]
if final_text_responses:
    print("\n--- Final Response ---")
    print(final_text_responses[0]["content"]["parts"][0]["text"])

## Update Agent

In [ ]:
if 'resource_name' not in locals():
    REASONING_ENGINE_ID = os.environ["REASONING_ENGINE_ID"]
    PROJECT_NUMBER = os.environ["GOOGLE_CLOUD_PROJECT_NUMBER"]
    resource_name = f"projects/{PROJECT_NUMBER}/locations/us-central1/reasoningEngines/{REASONING_ENGINE_ID}"

print(f"Resource name: {resource_name}")
remote_agent = agent_engines.get(resource_name)

In [ ]:
from vertexai import agent_engines

agent_engines.update(
    resource_name=resource_name,    # Required.
    agent_engine=app,     # Optional.
    requirements="requirements.txt",      # Optional.
    display_name="natura",    # Optional.
    extra_packages=["agent.py"],  # Optional.
    env_vars=dotenv_values("../.env")
)

## Rode os códigos abaixo

python3 03_register_authorization_resource.py

python3 04_register_agentoauth.py